# Set
Crear un algoritmo que pueda reconocer conjuntos ganadores dentro de un set de cartas del juego *set*.
## Crear Matriz One-hot-encoding

Las cartas han sido etiquetadas según c/u de sus 4 características:
   1. forma
   2. color
   3. relleno
   4. número
   
Lo primero que tenemos que hacer es representar cada una de estas cartas por medio de una matriz binaria donde se pondrá un 0 si no cumplen con una característica y 1 si sí la cumplen.  
La matriz cuenta con 12 columnas en total (3 tipos de cada característica)

In [1]:
import os
import pandas as pd
import numpy as np

Leer archivo que contiene todas las fotos del juego, crear un df según su forma, número, color y fondo.

In [2]:
cards = [x for x in os.listdir("cartas_clasificadas/") if "(" not in x]

In [4]:
cards = (
    pd.Series(cards)
    .str.extract("(?P<forma>\w+)-(?P<numero>\w+)-(?P<color>\w+)-(?P<fondo>\w+)")
)
cards_onehot = pd.get_dummies(cards)

In [5]:
cards_onehot.loc[:3]

,forma_churro,forma_ovalo,forma_rombo,numero_dos,numero_tres,numero_uno,color_morado,color_rojo,color_verde,fondo_blanco,fondo_rayado,fondo_solido
0,1,0,0,1,0,0,1,0,0,1,0,0
1,1,0,0,1,0,0,1,0,0,0,1,0
2,1,0,0,1,0,0,1,0,0,0,0,1
3,1,0,0,1,0,0,0,1,0,1,0,0


In [9]:
cards_onehot.shape

(81, 12)

## Modelo

Tomaremos un subconjunto de `n_cards` cartas. Este representará las cartas que se encuentran en el tablero.  


In [10]:
X=cards_onehot.sample(4, random_state=99912)

In [7]:
def Set(X):
    """
    Function that given an array of cards find sets.
    Input:
        X: np.array
            Array de cartas en el tablero donde
            X.shape = (n_cartas,n_features)
            Por definición del juego n_features simempre es igual a 12
    Output:
        R: tuple de np.arrays
            Cada tuple se lee de manera vertical y representa el índice
            de las cartas ganadores. Este indice es la representacion de enumerar 
            las cartas de 0 a n cartas.
    """
    n_cards = X.shape[0]
    # Expandir Matriz 
    X2 = X.reshape((n_cards,1,12))
    # Primera comparacion
    # Encontramos coincidencias 2 a 2
    X3 = X2 + X
    ## Nuevamente expandimos
    X4 = X3.reshape((n_cards,n_cards,1,12))
    ## Comparacion 3 a 3
    X4 =  X4 + X
    ## Creamos matriz filtro
    ## Esta matriz nos indica los conjuntos
    ## donde dos condiciones son igues y una no
    X5 = (X4==2).sum(axis=3)
    # Estos son los índices ganadores!
    # Los índices donde no hay 2
    R = (X5==0).nonzero()
    return R

In [63]:
def prune_results(results):
    """
    Once we found the winner indices 
    remove diagonal and remove duplicates
    Input
    ------
        results: tuple of arrays
            representing the indices of the cards that are a "set"
    Output
    ------
        res: np.array
            where each row represent the index of a "set" card
    """
    res = np.array(results)
    ## Remove duplicates
    ## (i.e. mean(column)!= max(column))
        # Find non-equal column indices
    non_equal = np.apply_along_axis(lambda col: max(col)!=np.mean(col),0,res)
        # Select non equal columns
    res= res[:, non_equal]
    ## remove duplicates
        ## Sort values and transpose
    res = np.apply_along_axis(sorted,0,res).T
        ## Get unique values
    res = np.unique(res,axis=0)
    return res

Se selecciona un set de 12 cartas y se buscan conjuntos

In [83]:
## Cartas
n_cards = 12
# Seleccionar cartas de nuestra base de cartas al azar
selected_cards = cards_onehot.sample(n_cards, random_state=99912)
## Convertir a np.array
X= selected_cards.to_numpy()

In [79]:
## Aplicar modelo
gan = Set(X)
## Limpiar indices
results = prune_results(gan)

In [80]:
## Los resultados dicen que se encontraron 3 sets
## (Cada renglon es un set)
results

array([[ 0,  2,  8],
       [ 0, 10, 11],
       [ 4,  7, 11]], dtype=int64)

In [84]:
# Las cartas en el índice 0,2 y 8 representan un set
## Comprobación
selected_cards.iloc[[0,2,8]]

,forma_churro,forma_ovalo,forma_rombo,numero_dos,numero_tres,numero_uno,color_morado,color_rojo,color_verde,fondo_blanco,fondo_rayado,fondo_solido
3,1,0,0,1,0,0,0,1,0,1,0,0
66,0,0,1,0,1,0,0,1,0,1,0,0
48,0,1,0,0,0,1,0,1,0,1,0,0


In [85]:
# Las cartas en el índice 0,10 y 11 representan un set
## Comprobación
selected_cards.iloc[[0,10,11]]

,forma_churro,forma_ovalo,forma_rombo,numero_dos,numero_tres,numero_uno,color_morado,color_rojo,color_verde,fondo_blanco,fondo_rayado,fondo_solido
3,1,0,0,1,0,0,0,1,0,1,0,0
60,0,0,1,1,0,0,0,0,1,1,0,0
27,0,1,0,1,0,0,1,0,0,1,0,0


In [86]:
# Las cartas en el índice 0,10 y 11 representan un set
## Comprobación
selected_cards.iloc[[4,7,11]]

,forma_churro,forma_ovalo,forma_rombo,numero_dos,numero_tres,numero_uno,color_morado,color_rojo,color_verde,fondo_blanco,fondo_rayado,fondo_solido
35,0,1,0,1,0,0,0,0,1,0,0,1
31,0,1,0,1,0,0,0,1,0,0,1,0
27,0,1,0,1,0,0,1,0,0,1,0,0
